In [4]:
import pandas as pd       
train = pd.read_csv("hp.csv", header=0)#, \
                    #delimiter=",", quoting=3)
train = train[pd.notnull(train["Positive_Review"])]
train = train[pd.notnull(train["Hotel_Name"])]
#train=train.reset_index(drop=True)
train.head(3)

,Unnamed: 0,Hotel_Name,Positive_Review
0,0,Hotel Arena,Only the park outside of the hotel was beauti...
1,1,Hotel Arena,No real complaints the hotel was great great ...
2,2,Hotel Arena,Location was good and staff were ok It is cut...


In [5]:
hotel = pd.read_csv("hotel_list.csv", header=0)
hotel.head(3)

,Unnamed: 0,old_ind,Hotel_Name,Average_Score
0,0,0,Hotel Arena,7.7
1,1,405,K K Hotel George,8.5
2,2,971,Apex Temple Court Hotel,9.2


In [11]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    #review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
        # 5. WordNetLemmatize
    wnl = WordNetLemmatizer()
    tagged = nltk.pos_tag(meaningful_words)
    w7=meaningful_words
    for i in range(len(meaningful_words)):
        wntag = get_wordnet_pos(tagged[i][1])
        if wntag is None:# not supply tag in case of None
            lemma = wnl.lemmatize(meaningful_words[i]) 
        else:
            lemma = wnl.lemmatize(meaningful_words[i], pos=wntag) 
        meaningful_words[i]=lemma
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [14]:
clean_review = review_to_words( train["Positive_Review"][1] )
print (clean_review)

real complaint hotel great great location surroundings room amenity service two recommendation however firstly staff upon check confuse regard deposit payment staff offer upon checkout refund original payment make new one bit confuse secondly site restaurant bit lack well think excellent quality food anyone vegetarian vegan background even wrap toast sandwich option would great aside minor minor thing fantastic spot back return amsterdam


In [15]:
train["Positive_Review"][1]

' No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam '

In [17]:
import numpy as np
import pandas as pd       

train = pd.read_csv("hp.csv", header=0)
train = train[pd.notnull(train["Positive_Review"])]
train = train[pd.notnull(train["Hotel_Name"])]
# Get the number of reviews based on the dataframe column size
num_reviews = 405#train["reviews.text"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    # If the index is evenly divisible by 100, print a message
    if( (i+1)%100 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews )  )                                                                  
    clean_train_reviews.append( review_to_words( train["Positive_Review"][i] ))

Review 100 of 405

Review 200 of 405

Review 300 of 405

Review 400 of 405



In [18]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

print (train_data_features.shape)
print (train_data_features.dtype)

Creating the bag of words...

(405, 1090)
int64


In [19]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print (vocab)

[u'ability', u'absolutely', u'access', u'accessibility', u'accessible', u'accommodate', u'accommodation', u'actually', u'add', u'adjacent', u'adorable', u'advice', u'advise', u'age', u'agreeable', u'air', u'airport', u'airy', u'allow', u'almost', u'alongside', u'already', u'also', u'although', u'always', u'amaze', u'amazing', u'amazingly', u'ambience', u'amenity', u'american', u'amsterdam', u'another', u'ansterdams', u'answer', u'anyone', u'anything', u'anyways', u'anywhere', u'appreciate', u'appreciation', u'architecturally', u'architecture', u'area', u'arena', u'arenaaa', u'arent', u'argument', u'around', u'arrange', u'arrival', u'arrive', u'arrived', u'art', u'artsy', u'artwork', u'aside', u'ask', u'assume', u'athmosphere', u'atmosphere', u'attention', u'attentive', u'attic', u'attraction', u'available', u'average', u'avoid', u'away', u'awesome', u'awful', u'axcess', u'back', u'background', u'bacon', u'bad', u'bake', u'bakery', u'balloon', u'bar', u'basically', u'basrs', u'batch', u

In [41]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)
df = []
# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    #print (count, tag)
    if count>50:
        #print (count, tag)
        #tmp=[count, tag]
        df.append(tag)
print (df)    

[u'beautiful', u'bed', u'comfortable', u'friendly', u'good', u'great', u'helpful', u'hotel', u'location', u'nice', u'park', u'room', u'staff']


In [68]:
import numpy as np
import pandas as pd       
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv("hp.csv", header=0)
train = train[pd.notnull(train["Positive_Review"])]
train = train[pd.notnull(train["Hotel_Name"])]
# Get the number of reviews based on the dataframe column size
hotel=pd.read_csv("hotel_list.csv", header=0)
ss=[]
for j in range(1,len(hotel['Hotel_Name'])):
    num_reviews2 = hotel['old_ind'][j]
    num_reviews1 = hotel['old_ind'][j-1]
    #print(num_reviews2)
    # Initialize an empty list to hold the clean reviews
    clean_train_reviews = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the movie review list 
    for i in range( num_reviews1, num_reviews2 ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    # If the index is evenly divisible by 100, print a message
        #if( (i+1)%100 == 0 ):
            #print ("Review %d of %d\n" % ( i+1, num_reviews )  )                                                                  
        clean_train_reviews.append( review_to_words( train["Positive_Review"][i] ))
    if ( j%100 == 0 ):
        print (j,". Creating the bag of words...\n")

    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    # Take a look at the words in the vocabulary
    vocab = vectorizer.get_feature_names()
    # Sum up the counts of each vocabulary word
    dist = np.sum(train_data_features, axis=0)
    df = []
    df2=[]
    marker=0
    # For each, print the vocabulary word and the number of times it 
    # appears in the training set
    for tag, count in zip(vocab, dist):
        #print (count, tag)
        if count>50:
            #print (count, tag)
            #tmp=[count, tag]
            df.append(tag)
            marker=1
        elif count>25:
            df2.append(tag)
    
    #print (df) 
    if marker==1:
        ss.append(df)
    else:
        ss.append(df2) 
#print(ss)
hotel2=hotel
hotel2["key"]=hotel2["Hotel_Name"]
hotel2["key"][:1492]=ss
print("Done~")

(100, '. Creating the bag of words...\n')
(200, '. Creating the bag of words...\n')
(300, '. Creating the bag of words...\n')
(400, '. Creating the bag of words...\n')
(500, '. Creating the bag of words...\n')
(600, '. Creating the bag of words...\n')
(700, '. Creating the bag of words...\n')
(800, '. Creating the bag of words...\n')
(900, '. Creating the bag of words...\n')
(1000, '. Creating the bag of words...\n')
(1100, '. Creating the bag of words...\n')
(1200, '. Creating the bag of words...\n')
(1300, '. Creating the bag of words...\n')
(1400, '. Creating the bag of words...\n')
Done~


/home/rl/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
hotel2=hotel
len(hotel2["Average_Score"])

1493

In [70]:
hotel2[10:20]

,Unnamed: 0,old_ind,Hotel_Name,Average_Score,key
10,10,7743,Hotel Trianon Rive Gauche,8.2,"[good, great, hotel, location, room, staff]"
11,11,7961,InterContinental London Park Lane,8.5,"[location, staff]"
12,12,8121,Novotel Suites Paris Nord 18 me,7.4,[room]
13,13,8301,Grand Royale London Hyde Park,7.7,"[also, area, around, bar, bathroom, beautiful,..."
14,14,11259,Milestone Hotel Kensington,9.5,[staff]
15,15,11345,Crowne Plaza London Kings Cross,8.1,"[bathroom, bed, breakfast, clean, comfortable,..."
16,16,12352,Select Hotel,8.9,"[good, great, hotel, location, room, staff]"
17,17,12608,H tel De Vend me,8.7,"[location, staff]"
18,18,12697,Pullman Paris Centre Bercy,8.7,"[good, great, hotel, room, staff]"
19,19,12921,Gardette Park Hotel,8.2,"[good, hotel, location, nice, room, staff]"


In [71]:
hotel2.to_csv("hotel+tag2.csv")